In [1]:
import time
a = time.time()
!wget https://s3-ap-southeast-1.amazonaws.com/he-public-data/DL%23+Beginner.zip
b = time.time()
!7z x 'DL#+Beginner.zip'
!rm 'DL#+Beginner.zip'
c = time.time()

--2020-08-02 05:37:35--  https://s3-ap-southeast-1.amazonaws.com/he-public-data/DL%23+Beginner.zip
Resolving s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)... 52.219.40.234
Connecting to s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)|52.219.40.234|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6873530111 (6.4G) [application/zip]
Saving to: ‘DL#+Beginner.zip’

DL#+Beginner.zip    100%[===================>]   6.40G  10.7MB/s    in 10m 40s 

2020-08-02 05:48:16 (10.2 MB/s) - ‘DL#+Beginner.zip’ saved [6873530111/6873530111]


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 6873530111 bytes (6556 MiB)

Extracting archive: DL#+Beginner.zip

ERRORS:
Headers Error

--
Path = DL#+Beginner.zip
Type = zip
ERROR

In [2]:
cd 'DL# Beginner'

/content/DL# Beginner


In [3]:
!unzip -qq test.zip
!rm test.zip

!7z x train.zip > nul
!rm train.zip

!unzip -qq meta-data.zip
!rm meta-data.zip
d = time.time()


ERRORS:
Headers Error



In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import keras as keras
from keras.layers import *
from keras.models import Model, Sequential, load_model
from keras.activations import *
from scipy import ndimage
import cv2
from PIL import Image
import time
import tensorflow
import json
import os
import tensorflow as tf
from sklearn.metrics import mean_squared_error
from math import sqrt
from skimage.transform import rescale, resize, downscale_local_mean
from skimage import data, color
import random
import keras.optimizers
import math

Using TensorFlow backend.


In [5]:
test = pd.read_csv('meta-data/test.csv')
train = pd.read_csv('meta-data/train.csv')
sample = pd.read_csv('meta-data/sample_submission.csv')
ytrain = pd.get_dummies(train.Animal)

In [ ]:
train.Animal

0           hippopotamus
1               squirrel
2           grizzly+bear
3                     ox
4        german+shepherd
              ...       
12995                bat
12996                 ox
12997           squirrel
12998           squirrel
12999           squirrel
Name: Animal, Length: 13000, dtype: object

In [ ]:
from keras.utils import Sequence
import math

class data_gen(Sequence):
  
  def __init__(self, labels, batch_size):
    self.target = labels
    self.batch_length = batch_size
    self.size = len(labels)

  def __getitem__(self, idx):
    start = idx*self.batch_length
    end = min((idx+1)*self.batch_length, self.size)
    
    lis = []
    for i in range(start,end):
      img = cv2.imread('train/Img-'+str(i+1)+'.jpg')
      
      if len(img.shape)==2:
            img = np.stack((img,)*3, axis=-1)
          
      #scaling
      (l,b,n_c) = img.shape
      mini = min(l,b)
      fraction = 224.0/mini
      img_scaled = cv2.resize( img , ( int((b/mini)*224) , int((l/mini)*224)  )  )
      
      #cropping
      (l,b,n_c) = img_scaled.shape
      ran = random.randint(0,1)
      if l>b:
        img_scaled_crpd = img_scaled[ (l-224)*ran + 0  : (l-224)*ran  + 224 , : ]
      else:
        img_scaled_crpd = img_scaled[ :, (b-224)*ran + 0  : (b-224)*ran  + 224 ]
        
      #flipping  
      if random.randint(0,1):
            img_scaled_crpd = img_scaled_crpd[:,::-1]
          
      lis.append(img_scaled_crpd/255.0)
    lis = np.array( lis )
  
    
    
    return lis, self.target[start:end]

  def __len__(self):
    return math.ceil(self.size / self.batch_length)

In [ ]:
!pip install efficientnet

In [ ]:
import efficientnet.keras as efn

In [ ]:
base_model = efn.EfficientNetB0( weights='imagenet', include_top = False )

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
predictions = Dense(30, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, None, None, 3 864         input_2[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, None, None, 3 128         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, None, None, 3 0           stem_bn[0][0]                    
____________________________________________________________________________________________

In [ ]:
len(model.layers)

233

In [ ]:
i=0
for layer in base_model.layers:
    layer.trainable = False
    i = i+1
    if (i>230): break

In [ ]:
model.compile(  loss="categorical_crossentropy", optimizer='adam',metrics=["accuracy"])

In [ ]:
num_classes  = 30
train_size = 13000
batch_size = 64

In [ ]:
model.fit_generator(
        data_gen( ytrain , batch_size ),
        steps_per_epoch=int( train_size/batch_size ),
        epochs=10
        )

Epoch 1/10
101/101 [==============================] - 201s 2s/step - loss: 1.7911 - accuracy: 0.5950
Epoch 2/10
101/101 [==============================] - 199s 2s/step - loss: 0.8486 - accuracy: 0.7990
Epoch 3/10
101/101 [==============================] - 199s 2s/step - loss: 0.6460 - accuracy: 0.8376
Epoch 4/10
101/101 [==============================] - 197s 2s/step - loss: 0.5696 - accuracy: 0.8509
Epoch 5/10
101/101 [==============================] - 198s 2s/step - loss: 0.5157 - accuracy: 0.8646
Epoch 6/10
101/101 [==============================] - 197s 2s/step - loss: 0.4628 - accuracy: 0.8750
Epoch 7/10
101/101 [==============================] - 196s 2s/step - loss: 0.4346 - accuracy: 0.8812
Epoch 8/10
101/101 [==============================] - 197s 2s/step - loss: 0.4138 - accuracy: 0.8866
Epoch 9/10
101/101 [==============================] - 196s 2s/step - loss: 0.4046 - accuracy: 0.8848
Epoch 10/10
101/101 [==============================] - 197s 2s/step - loss: 0.3729 - accura

In [ ]:
from keras.models import save_model
model.save('model.h5')
#-----------------

In [ ]:
model = load_model( '../model.h5' )

In [ ]:
for layer in model.layers:
    layer.trainable = True

In [ ]:
model.compile(  loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=0.001),
              metrics=["accuracy"])

In [ ]:
model.fit_generator(
        data_gen( ytrain , batch_size ),
        steps_per_epoch=int( train_size/batch_size ),
        epochs=5
        )

Epoch 1/5
203/203 [==============================] - 396s 2s/step - loss: 0.4793 - accuracy: 0.8496
Epoch 2/5
203/203 [==============================] - 372s 2s/step - loss: 0.2706 - accuracy: 0.9116
Epoch 3/5
203/203 [==============================] - 368s 2s/step - loss: 0.1776 - accuracy: 0.9422
Epoch 4/5
203/203 [==============================] - 369s 2s/step - loss: 0.1645 - accuracy: 0.9486
Epoch 5/5
203/203 [==============================] - 366s 2s/step - loss: 0.1264 - accuracy: 0.9594


In [ ]:
#-----------------

In [ ]:
from keras.utils import Sequence
import math

class predict_gen(Sequence):
  
  def __init__(self, tup , batch_size):
    self.tup = tup
    self.batch_length = batch_size
    self.size = 6000

  def __getitem__(self, idx):
    start = idx*self.batch_length
    end = min((idx+1)*self.batch_length, self.size)
    (one,two) = self.tup
    
    lis = []
    for i in range(start,end):
      img = cv2.imread('test/Img-'+str(i+1)+'.jpg')
      
      if len(img.shape)==2:
            img = np.stack((img,)*3, axis=-1)
          
      #scaling
      (l,b,n_c) = img.shape
      mini = min(l,b)
      fraction = 224.0/mini
      img_scaled = cv2.resize( img , ( int((b/mini)*224) , int((l/mini)*224)  )  )
      
      #cropping
      (l,b,n_c) = img_scaled.shape
      ran = one
      if l>b:
        img_scaled_crpd = img_scaled[ (l-224)*ran + 0  : (l-224)*ran  + 224 , : ]
      else:
        img_scaled_crpd = img_scaled[ :, (b-224)*ran + 0  : (b-224)*ran  + 224 ]
        
      #flipping  
      if two:
            img_scaled_crpd = img_scaled_crpd[:,::-1]
          
      lis.append(img_scaled_crpd/255.0)
    lis = np.array( lis )
  
    
    
    return lis
  def __len__(self):
    return math.ceil(self.size / self.batch_length)

In [ ]:
y1 = model.predict_generator( predict_gen( (0,0),64) )
y2 = model.predict_generator( predict_gen( (0,1),64) )
y3 = model.predict_generator( predict_gen( (1,0),64) )
y4 = model.predict_generator( predict_gen( (1,1),64) )

In [ ]:
y = (y1 + y2 + y3 + y4)/4.0

In [ ]:
sample.iloc[:,1:] = y

In [ ]:
sample.head()

,image_id,antelope,bat,beaver,bobcat,buffalo,chihuahua,chimpanzee,collie,dalmatian,german+shepherd,grizzly+bear,hippopotamus,horse,killer+whale,mole,moose,mouse,otter,ox,persian+cat,raccoon,rat,rhinoceros,seal,siamese+cat,spider+monkey,squirrel,walrus,weasel,wolf
0,Img-1.jpg,4.659089e-06,1.059411e-08,2.373346e-06,1.254051e-07,9.985394e-01,4.607892e-11,6.721491e-10,7.939410e-09,1.414501e-09,2.676140e-09,5.918792e-04,7.948282e-05,3.696219e-06,6.881829e-09,2.834688e-07,2.643670e-05,9.098905e-08,5.371133e-06,2.538309e-04,5.185120e-10,6.190286e-08,1.482894e-07,1.230171e-04,3.443259e-04,2.020510e-10,7.654829e-10,5.418551e-10,2.468076e-05,2.772706e-08,3.674053e-10
1,Img-2.jpg,1.325346e-06,3.599529e-04,4.655784e-05,1.365560e-02,2.659452e-05,2.915471e-02,9.346968e-04,1.910049e-05,6.208041e-05,2.583370e-02,7.961073e-05,9.220761e-02,5.970773e-04,4.671122e-02,2.469349e-04,5.427339e-06,2.768449e-03,1.026379e-04,3.023060e-05,6.658754e-02,2.846962e-03,9.289112e-03,4.240035e-03,2.417198e-02,6.505201e-01,2.749918e-04,1.186030e-02,1.325112e-02,1.475764e-04,3.966826e-03
2,Img-3.jpg,3.838302e-03,5.992521e-06,1.024517e-05,6.963611e-05,9.292790e-05,1.571868e-07,2.256216e-06,8.446293e-06,1.638201e-06,3.688199e-05,4.239383e-05,7.147198e-04,1.252533e-04,1.202771e-05,1.511343e-06,9.929831e-01,5.195969e-07,8.651727e-06,1.558049e-06,3.529781e-07,4.077012e-07,3.176947e-06,2.003696e-03,3.816685e-06,3.728011e-07,5.053294e-07,2.136348e-07,1.353859e-07,6.089295e-06,2.503206e-05
3,Img-4.jpg,4.818286e-07,1.834266e-08,6.074005e-10,1.518872e-08,6.484154e-11,1.213953e-05,3.067240e-08,2.612715e-08,2.829059e-08,4.910760e-08,3.051844e-10,2.114458e-09,8.540264e-11,1.206661e-09,2.229436e-10,1.262129e-10,3.153440e-09,9.534176e-11,1.469198e-10,3.320017e-09,9.999871e-01,4.571880e-10,1.136988e-11,2.439246e-11,6.940108e-09,4.831052e-09,1.260955e-08,5.500875e-11,7.247531e-08,4.934227e-10
4,Img-5.jpg,6.428571e-08,5.841888e-01,8.663654e-03,3.009140e-06,1.763711e-06,1.591147e-05,3.993536e-05,1.101661e-03,1.100355e-07,8.009147e-02,1.849256e-05,1.050813e-06,4.216169e-05,5.201300e-05,3.086916e-01,7.883577e-06,3.576262e-03,2.431737e-04,5.504047e-07,3.283630e-03,3.956437e-03,4.820331e-03,4.918018e-08,1.124407e-05,2.227886e-05,4.223519e-05,5.755515e-04,6.659878e-07,5.331436e-04,1.500956e-05


In [ ]:
sample.to_csv('first.csv', index = False)

In [ ]:
model.save('model.h5')